# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission, if necessary. Sections that begin with **'Implementation'** in the header indicate where you should begin your implementation for your project. Note that some sections of implementation are optional, and will be marked with **'Optional'** in the header.

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---
## Step 0: Load The Data

In [1]:
# Load pickled data
import pickle
import numpy as np

# TODO: Fill this in based on where you saved the training and testing data

training_file = 'train.p'
testing_file = 'test.p'

def load_dataset(training_file, testing_file):
    with open(training_file, mode='rb') as f:
        train = pickle.load(f)
    with open(testing_file, mode='rb') as f:
        test = pickle.load(f)
    
    X_train, y_train = train['features'], train['labels']
    X_test, y_test = test['features'], test['labels']
    C_train, C_test = train['coords'], test['coords']
    return X_train, y_train, X_test, y_test, C_train, C_test

X_train, y_train, X_test, y_test, C_train, C_test = load_dataset(training_file, testing_file)

---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 1D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below.

In [2]:
### Replace each question mark with the appropriate value.

# TODO: Number of training examples
n_train = len(X_train)

# TODO: Number of testing examples.
n_test = len(X_test)

# TODO: What's the shape of an traffic sign image?
image_shape = X_train[0].shape

# TODO: How many unique classes/labels there are in the dataset.
n_classes = len(np.unique(y_train))

Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc.

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections.

In [3]:
### Data exploration visualization goes here.
### Feel free to use as many code cells as needed.
import random
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import csv
import cv2
# Visualizations will be shown in the notebook.
%matplotlib inline

"""
signames_file = 'signnames.csv'
def read_signames(signames_file):
    sign_names = []
    with open(signames_file) as f:
        reader = csv.DictReader(f)
        sign_names = [line['SignName'] for line in reader]
    return sign_names

sign_names = read_signames(signames_file)

index = random.randint(0, len(X_train))
image = X_train[index].squeeze()

plt.figure(figsize=(1,1))
plt.imshow(image, cmap='gray')
print(y_train[index])

hist, bins = np.histogram(y_train, bins=n_classes)
plt.figure()
plt.hist(y_train, bins=n_classes, color='#00ffCC')
plt.axis([0, 43, 0, 2500])
plt.show()

# plotting the count of each sign

y_pos = range(n_classes)
label_list = y_train.tolist()
sign_type = [label_list.count(y) for y in range(n_classes)]

plt.bar(y_pos, sign_type, width=0.8, align='center')
plt.ylabel('Sample Count')
plt.xlabel('Sample Class')
plt.show()

fig, axs = plt.subplots(2,5, figsize=(15, 6))
fig.subplots_adjust(hspace = .2, wspace=.001)
axs = axs.ravel()
for i in range(10):
    index = random.randint(0, len(X_train))
    image = X_train[index]
    axs[i].axis('off')
    axs[i].imshow(image)
    axs[i].set_title(y_train[index])
    
num_x = 5
num_y = 5
fig, ax = plt.subplots(num_y,num_x)
fig.set_size_inches(10,15)
for i in range(num_x*num_y):
    n = random.random()*n_train
    ax[int(i/num_x)][i%num_x].imshow(X_train[n])
    coords = C_train[n]
    rect = patches.Rectangle((coords[0],coords[1]),coords[2]-coords[0],coords[3]-coords[1],edgecolor='#00ff00',facecolor='none')
    ax[int(i/num_x)][i%num_x].add_patch(rect)
    ax[int(i/num_x)][i%num_x].set_title(sign_names[y_train[n]], fontsize=8)

plt.show()

# show image of N random data points
count = 10
fig, axs = plt.subplots(count, 3, figsize=(count, count*3))
fig.subplots_adjust(hspace = .2, wspace=.001)
axs = axs.ravel()
for i in range(0, count*3, 3):
    index = random.randint(0, len(X_train))
    image = X_train[index]
    
    axs[i].axis('off')
    axs[i].imshow(image)
    axs[i].set_title(sign_names[y_train[index]])

    bw = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    axs[i+1].axis('off')
    axs[i+1].imshow(bw, cmap='gray')
    axs[i+1].set_title("B/W")

    equ = cv2.equalizeHist(bw)
    axs[i+2].axis('off')
    axs[i+2].imshow(equ, cmap='gray')
    axs[i+2].set_title("Histogram Equalized")
"""    

'\nsignames_file = \'signnames.csv\'\ndef read_signames(signames_file):\n    sign_names = []\n    with open(signames_file) as f:\n        reader = csv.DictReader(f)\n        sign_names = [line[\'SignName\'] for line in reader]\n    return sign_names\n\nsign_names = read_signames(signames_file)\n\nindex = random.randint(0, len(X_train))\nimage = X_train[index].squeeze()\n\nplt.figure(figsize=(1,1))\nplt.imshow(image, cmap=\'gray\')\nprint(y_train[index])\n\nhist, bins = np.histogram(y_train, bins=n_classes)\nplt.figure()\nplt.hist(y_train, bins=n_classes, color=\'#00ffCC\')\nplt.axis([0, 43, 0, 2500])\nplt.show()\n\n# plotting the count of each sign\n\ny_pos = range(n_classes)\nlabel_list = y_train.tolist()\nsign_type = [label_list.count(y) for y in range(n_classes)]\n\nplt.bar(y_pos, sign_type, width=0.8, align=\'center\')\nplt.ylabel(\'Sample Count\')\nplt.xlabel(\'Sample Class\')\nplt.show()\n\nfig, axs = plt.subplots(2,5, figsize=(15, 6))\nfig.subplots_adjust(hspace = .2, wspace=.00

----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

There are various aspects to consider when thinking about this problem:

- Neural network architecture
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

**NOTE:** The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play!

### Implementation

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [4]:
### Preprocess the data here.
### Feel free to use as many code cells as needed.
from sklearn.utils import shuffle
import scipy.ndimage
import numpy as np
"""
def augment_brightness_images(image):
    image1 = cv2.cvtColor(image,cv2.COLOR_RGB2HSV)
    random_bright = .25+np.random.uniform()
    #print(random_bright)
    image1[:,:,2] = image1[:,:,2]*random_bright
    image1 = cv2.cvtColor(image1,cv2.COLOR_HSV2RGB)
    return image1

def create_variant(image):
    if (random.choice([True, False])):
        image = scipy.ndimage.interpolation.shift(image, [random.randrange(-5, 5), random.randrange(-5, 5), 0])
    if (random.choice([True, False])):
        image = scipy.ndimage.interpolation.rotate(image, random.randrange(-15, 15), reshape=False)
    return image

# convert to B/W
X_train_bw = np.array([cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) for image in X_train])
X_test_bw = np.array([cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) for image in X_test])

# apply histogram equalization
X_train_hst_eq = np.array([cv2.equalizeHist(image) for image in X_train_bw])
X_test_hst_eq = np.array([cv2.equalizeHist(image) for image in X_test_bw])

# reshape for conv layer
#X_train = X_train_hst_eq[..., np.newaxis]
#X_test = X_test_hst_eq[..., np.newaxis]
#print('Before shaping:', X_train_hst_eq.shape)
#print('After shaping:', X_train.shape)

# normalize range
#X_train_normalized = (X_train_reshaped - np.mean(X_train_reshaped)) / 128.0
#X_test_normalized = (X_test_reshaped - np.mean(X_test_reshaped)) / 128.0
#print('Mean before normalizing:', np.mean(X_train_reshaped), np.mean(X_test_reshaped))
#print('Mean after normalizing:', np.mean(X_train_normalized), np.mean(X_test_normalized))
"""

"\ndef augment_brightness_images(image):\n    image1 = cv2.cvtColor(image,cv2.COLOR_RGB2HSV)\n    random_bright = .25+np.random.uniform()\n    #print(random_bright)\n    image1[:,:,2] = image1[:,:,2]*random_bright\n    image1 = cv2.cvtColor(image1,cv2.COLOR_HSV2RGB)\n    return image1\n\ndef create_variant(image):\n    if (random.choice([True, False])):\n        image = scipy.ndimage.interpolation.shift(image, [random.randrange(-5, 5), random.randrange(-5, 5), 0])\n    if (random.choice([True, False])):\n        image = scipy.ndimage.interpolation.rotate(image, random.randrange(-15, 15), reshape=False)\n    return image\n\n# convert to B/W\nX_train_bw = np.array([cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) for image in X_train])\nX_test_bw = np.array([cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) for image in X_test])\n\n# apply histogram equalization\nX_train_hst_eq = np.array([cv2.equalizeHist(image) for image in X_train_bw])\nX_test_hst_eq = np.array([cv2.equalizeHist(image) for image in X_te

### Question 1 

_Describe how you preprocessed the data. Why did you choose that technique?_

**Answer:**

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

### Generate additional data (OPTIONAL!)
REQ_NUM_SAMPLES = 1500
generated_features = []
generated_labels = []
""""
for class_index in range(len(sign_type)):
    class_sample_count = sign_type[class_index]
    augment_multiple = round(REQ_NUM_SAMPLES / class_sample_count)
    print(class_index, class_sample_count, augment_multiple)

    if augment_multiple <= 1:
        continue
    
    for test_feature, test_label in zip(X_train_normalized, y_train):
        if class_index == test_label:
            for augment_iter in range(augment_multiple):
                generated_features.append(create_variant(test_feature))
                generated_labels.append(test_label)
"""
# append generated data to original data
#X_train_augmented = np.append(np.array(X_train_normalized), np.array(generated_features), axis=0)
#y_train_augmented = np.append(np.array(y_train), np.array(generated_labels), axis=0)
X_train, y_train = shuffle(X_train, y_train)

#y_pos = range(n_classes)
#label_list = y_train_augmented.tolist()
#sign_type = [label_list.count(y) for y in range(n_classes)]

#plt.bar(y_pos, sign_type, width=0.8, align='center')
#plt.ylabel('Sample Count')
#plt.xlabel('Sample Class')
#plt.show()

### and split the data into training/validation/testing sets here.
### Feel free to use as many code cells as needed.
# split the training set into training set and validation set
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.20, random_state=7)

                                                                # validation examples
n_validation = len(X_validation)

print("Number of training examples =", n_train)
print("Number of validation examples =", n_validation)
print("Number of testing examples =", n_test)
print("Image data shape =", X_train.shape)
print("Number of classes =", n_classes)
                                                                
assert(len(X_train) == len(y_train))
assert(len(X_validation) == len(y_validation))
assert(len(X_test) == len(y_test))                                                                

Number of training examples = 39209
Number of validation examples = 7842
Number of testing examples = 12630
Image data shape = (31367, 32, 32, 3)
Number of classes = 43


### Question 2

_Describe how you set up the training, validation and testing data for your model. **Optional**: If you generated additional data, how did you generate the data? Why did you generate the data? What are the differences in the new dataset (with generated data) from the original dataset?_

**Answer:**

In [5]:
### Define your architecture here.
### Feel free to use as many code cells as needed.
import tensorflow as tf

EPOCHS = 10
BATCH_SIZE = 128

### Question 3

_What does your final architecture look like? (Type of model, layers, sizes, connectivity, etc.)  For reference on how to build a deep neural network using TensorFlow, see [Deep Neural Network in TensorFlow
](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/b516a270-8600-4f93-a0a3-20dfeabe5da6/concepts/83a3a2a2-a9bd-4b7b-95b0-eb924ab14432) from the classroom._


**Answer:**

In [4]:
import tensorflow as tf
from tensorflow.contrib.layers import flatten

EPOCHS = 30
BATCH_SIZE = 256

tf.set_random_seed(0.0)
# variable learning rate
lr = tf.placeholder(tf.float32)
# test flag for batch norm
tst = tf.placeholder(tf.bool)
iter = tf.placeholder(tf.int32)
# dropout probability
pkeep = tf.placeholder(tf.float32)

"""
class Layer(object):
    def __init__(self, 
                 input_channels = None, 
                 krows=None, 
                 kcols=None, 
                 output_channels = None,
                 strides=None):
        self.input_channels = input_channels
        self.krows = krows
        self.kcols = kcols
        self.output_channels = output_channels
        self.strides = [1, strides, strides, 1]
    
class Conv2d(Layer):
    def __init__(self, 
                 output_channels, 
                 nb_rows, 
                 nb_cols,
                 strides=1,
                 input_channels = None
                 input_shape = None):
        if input_shape:
            Layer.__init__(self, 
                           input_channels = input_shape[2], 
                           krows = krows,
                           kcols = kcols,
                           output_channels = output_channels,
                           strides = strides)
        elif input_channels:
            Layer.__init__(self, 
                           input_channels = input_channels, 
                           krows = krows,
                           kcols = kcols,
                           output_channels = output_channels,
                           strides = strides)
        else:
            raise ValueError("both input_channels and input_shape can't be None")
            
        #self.B = tf.Variable(tf.constant(0.1, tf.float32, [features]))
        self.Y = None
        if input_shape:
            X = tf.placeholder(tf.float32, (None,) + input_shape)
            self.W = tf.Variable(tf.truncated_normal(shape = input_shape + output_channels, 
                                                     mean = 0, 
                                                     stddev = 0.1))
            self.B = tf.Variable(tf.zeros(output_channels))            
            self.Y = tf.nn.conv2d(X, self.W, strides=self.strides, padding='VALID') + self.B
            self.Y = self.activation(self.Y)

    def Y(self, X=None, input_channels=None):
        if self.Y:
            pass
        elif X and input_channels:
            self.X = X
            self.input_channels = input_channels
            self.W = tf.Variable(tf.truncated_normal(shape = (krows, kcols, input_channels, output_channels), 
                                                     mean = 0, 
                                                     stddev = 0.1))
            self.B = tf.Variable(tf.zeros(output_channels))
            self.Y = tf.nn.conv2d(X, self.W, strides=self.strides, padding='VALID') + self.B
            self.Y = self.activation(self.Y)            
        return self.Y
    
    def layer_type(self):
        "conv"
        
class MaxPool(object):
    def __init__(self,
                 pool_size = pool_size, 
                 strides = strides):
        Layer.__init__(self, krows=pool_size[0], kcols=pool_size[1], strides=strides)
        self.Y = None
        
    def Y(self, X=None):
        if self.Y:
            pass
        elif X:
            self.X = X
            self.Y = tf.nn.max_pool(X, 
                                    ksize=[1, self.krows, self.kcols, 1] 
                                    strides=self.strides, 
                                    padding='VALID')        
        return self.Y
    
    def layer_type(self):
        return "pool"

class Flatten(Layer):
    def __init__(self):
        Layer.__init__(self)
        self.Y = None
    
    def Y(self, X=None)
        if self.Y:
            pass
        elif X:
            self.X = X
            self.Y = flatten(X)
        return self.Y
    
    def layer_type(self):
        return "flatten"
    
class FC(layer):
    def __init__(self, 
                 in_dim = in_dim, 
                 out_dim = out_dim,
                 activation = tf.nn.relu,
                 mean = 0,
                 stddev = 0.1
                 pkeep = None):
        self.input = x
        self.in_dim = in_dim
        self.out_dim = out_dim
        self.activation = activation
        self.pkeep = pkeep
        self.W = tf.Variable(tf.truncated_normal(shape=(in_dim, out_dim), 
                                                 mean = mean, 
                                                 stddev = stddev))
        self.B = tf.Variable(tf.zeros(features))
        self.output = tf.matmul(self.X, self.W) + self.B
        self.output = self.activation(Y)
    
    def Y(self, X=None, ):
        self.W = tf.Variable(tf.truncated_normal(shape=(in_dim, out_dim), 
                                                 mean = mean, 
                                                 stddev = stddev))
        self.B = tf.Variable(tf.zeros(features))
        self.output = tf.matmul(self.X, self.W) + self.B
        self.output = self.activation(Y)
        return self.Y
    
    def layer_type(self):
        return "fc"
        
class ConvNetModel(object):
    def __init__(self, 
                 model = model,
                 train = train, 
                 validation = validation, 
                 test = test):
        self._train = train
        self._validation = validation
        self._test = test
        self._last_layer = None
        self._flatten_dim = None
    
    def add(self, layer):
        if (self._last_layer):
            if self._last_layer.get_type() == 'conv':
                if self.layer.get_type() == 'conv':
                    Y = self.layer.Y(X=self._last_layer.Y(), 
                                     input_channels=self._last_layer.output_channels)
                elif self.layer.get_type() == 'pool':
                    Y = self.layer.Y(X=self._last_layer.Y())
                    self.layer.input_channels = self._last_layer.output_channels
                    self.layer.output_channels = self._last_layer.output_channels
                    self.krows = self._last_layer.krows
                    self.kcols = self._last_layer.kcols                    
                elif self.layer.get_type() == 'flatten':
                    self._flatten_dim = self._last_layer.krows * self._last_layer.kcols * self._last_layer.output_channels 
                else:
                    raise ValueError("invalild layer type " + self.layer.get_type())
            elif self._last_layer.get_type() == 'pool':
                    if self.layer.get_type() == 'conv':
                        Y = self.layer.Y(X=self._last_layer.Y(), 
                                         input_channels=self._last_layer.output_channels)
                     
        self._last_layer = self.layer
    
    def _get_last_layer_Y(self):
        if (self._last_layer):
            return self._last_layer.Y()
        pass
    
    def _get_last_layer_channels(self):
        if (self._last_layer):
            return self._last_layer.get_output_channels()
        return None
            
                    
model = ConvNetModel()
model.add(Conv2d(6, 5, 5, input_shape = (32, 32, 1), strides=1))
model.add(MaxPool(pool_size=(2,2), strides=2))
model.add(Conv2d(16, 5, 5))
model.add(MaxPool(ksize=[1,2,2,1], strides=2))
model.add(flatten())
model.add(FC(in_dim = 400, out_dim = 120))
model.add(FC(y, in_dim = 120, out_dim = 84))
model.add(FC(y, in_dim = 84, out_dim = 43))
"""

def LeNet(x):    
    # Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
    mu = 0
    sigma = 0.1
    
    # SOLUTION: Layer 1: Convolutional. Input = 32x32x1. Output = 28x28x6.
    conv1_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 3, 6), mean = mu, stddev = sigma))
    conv1_b = tf.Variable(tf.zeros(6))
    conv1   = tf.nn.conv2d(x, conv1_W, strides=[1, 1, 1, 1], padding='VALID') + conv1_b
    #conv1bn, update_ema1 = batchnorm(conv1, tst, iter, convolutional=True)

    # SOLUTION: Activation.
    conv1 = tf.nn.relu(conv1)

    # SOLUTION: Pooling. Input = 28x28x6. Output = 14x14x6.
    conv1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

    # SOLUTION: Layer 2: Convolutional. Output = 10x10x16.
    conv2_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 6, 16), mean = mu, stddev = sigma))
    conv2_b = tf.Variable(tf.zeros(16))
    conv2   = tf.nn.conv2d(conv1, conv2_W, strides=[1, 1, 1, 1], padding='VALID') + conv2_b
    #conv2bn, update_ema2 = batchnorm(conv2, tst, iter, convolutional=True)

    
    # SOLUTION: Activation.
    conv2 = tf.nn.relu(conv2)

    # SOLUTION: Pooling. Input = 10x10x16. Output = 5x5x16.
    conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

    # SOLUTION: Flatten. Input = 5x5x16. Output = 400.
    fc0   = flatten(conv2)
    
    # SOLUTION: Layer 3: Fully Connected. Input = 400. Output = 120.
    fc1_W = tf.Variable(tf.truncated_normal(shape=(400, 120), mean = mu, stddev = sigma))
    fc1_b = tf.Variable(tf.zeros(120))
    fc1   = tf.matmul(fc0, fc1_W) + fc1_b
    #fc1bn, update_ema3 = batchnorm(fc1, tst, iter)

    
    # SOLUTION: Activation.
    fc1    = tf.nn.relu(fc1)

    # SOLUTION: Layer 4: Fully Connected. Input = 120. Output = 84.
    fc2_W  = tf.Variable(tf.truncated_normal(shape=(120, 84), mean = mu, stddev = sigma))
    fc2_b  = tf.Variable(tf.zeros(84))
    fc2    = tf.matmul(fc1, fc2_W) + fc2_b
    #fc2bn, update_ema4 = batchnorm(fc2, tst, iter)
    
    # SOLUTION: Activation.
    fc2    = tf.nn.relu(fc2)
    fc2d   = tf.nn.dropout(fc2, pkeep)

    # SOLUTION: Layer 5: Fully Connected. Input = 84. Output = 10.
    fc3_W  = tf.Variable(tf.truncated_normal(shape=(84, 43), mean = mu, stddev = sigma))
    fc3_b  = tf.Variable(tf.zeros(43))
    logits = tf.matmul(fc2d, fc3_W) + fc3_b
    
    return logits

x = tf.placeholder(tf.float32, (None, 32, 32, 3))
y = tf.placeholder(tf.int32, (None))
one_hot_y = tf.one_hot(y, 43)
rate = 0.001

logits = LeNet(x)
Y = tf.nn.softmax(logits)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits, one_hot_y)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)

correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y, tst: True, pkeep: 0.75})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train)
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        X_train, y_train = shuffle(X_train, y_train)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y, tst: True, pkeep: 0.75})
            
        validation_accuracy = evaluate(X_validation, y_validation)
        print("EPOCH {} ...".format(i+1))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        print()
        
    saver.save(sess, './lenet')
    print("Model saved")

Training...

EPOCH 1 ...
Validation Accuracy = 0.281

EPOCH 2 ...
Validation Accuracy = 0.528

EPOCH 3 ...
Validation Accuracy = 0.677

EPOCH 4 ...
Validation Accuracy = 0.748

EPOCH 5 ...
Validation Accuracy = 0.810

EPOCH 6 ...
Validation Accuracy = 0.840

EPOCH 7 ...
Validation Accuracy = 0.867

EPOCH 8 ...
Validation Accuracy = 0.884

EPOCH 9 ...
Validation Accuracy = 0.896

EPOCH 10 ...
Validation Accuracy = 0.908

EPOCH 11 ...
Validation Accuracy = 0.903

EPOCH 12 ...
Validation Accuracy = 0.919

EPOCH 13 ...
Validation Accuracy = 0.925

EPOCH 14 ...
Validation Accuracy = 0.930

EPOCH 15 ...
Validation Accuracy = 0.933

EPOCH 16 ...
Validation Accuracy = 0.935

EPOCH 17 ...
Validation Accuracy = 0.932

EPOCH 18 ...
Validation Accuracy = 0.939

EPOCH 19 ...
Validation Accuracy = 0.942

EPOCH 20 ...
Validation Accuracy = 0.947

EPOCH 21 ...
Validation Accuracy = 0.944

EPOCH 22 ...
Validation Accuracy = 0.950

EPOCH 23 ...
Validation Accuracy = 0.955

EPOCH 24 ...
Validation Accura

### Question 4

_How did you train your model? (Type of optimizer, batch size, epochs, hyperparameters, etc.)_


**Answer:**

### Question 5


_What approach did you take in coming up with a solution to this problem? It may have been a process of trial and error, in which case, outline the steps you took to get to the final solution and why you chose those steps. Perhaps your solution involved an already well known implementation or architecture. In this case, discuss why you think this is suitable for the current problem._

**Answer:**

---

## Step 3: Test a Model on New Images

Take several pictures of traffic signs that you find on the web or around you (at least five), and run them through your classifier on your computer to produce example results. The classifier might not recognize some local signs but it could prove interesting nonetheless.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Implementation

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [9]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))

    test_accuracy = evaluate(X_test_normalized, y_test)
    print("Test Accuracy = {:.3f}".format(test_accuracy))

Test Accuracy = 0.904


### Question 6

_Choose five candidate images of traffic signs and provide them in the report. Are there any particular qualities of the image(s) that might make classification difficult? It could be helpful to plot the images in the notebook._



**Answer:**

In [ ]:
### Run the predictions here.
### Feel free to use as many code cells as needed.

### Question 7

_Is your model able to perform equally well on captured pictures when compared to testing on the dataset? The simplest way to do this check the accuracy of the predictions. For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate._

_**NOTE:** You could check the accuracy manually by using `signnames.csv` (same directory). This file has a mapping from the class id (0-42) to the corresponding sign name. So, you could take the class id the model outputs, lookup the name in `signnames.csv` and see if it matches the sign from the image._


**Answer:**

In [ ]:
### Visualize the softmax probabilities here.
### Feel free to use as many code cells as needed.

### Question 8

*Use the model's softmax probabilities to visualize the **certainty** of its predictions, [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. Which predictions is the model certain of? Uncertain? If the model was incorrect in its initial prediction, does the correct prediction appear in the top k? (k should be 5 at most)*

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

**Answer:**

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.